# Libraries

In [60]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, desc

# Start Spark session

In [61]:
spark = SparkSession.builder \
    .appName("spark_data_analysis") \
    .getOrCreate()

In [62]:
spark

# Data load

In [63]:
df = spark.read.csv(
    "./data.csv", 
    header=True,
    inferSchema=True
)

In [64]:
df.show(5)

+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  false|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  false|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  false|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 5 rows



In [65]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)



# Helper column

## match_resul

In [66]:
# The match_resul column will be created to indicate the result of the match, the results of this column will be:
# - home_team : When the value of home_score column is > than away_score column
# - away_team : When the value of away_score column is > than home_score column
# - draw : When the value of home_score column is == of away_score column

df = df.withColumn("match_resul", \
    when(df.home_score > df.away_score, lit("home_team")) \
    .when(df.away_score > df.home_score, lit("away_team")) \
    .otherwise(lit("draw")) \
    )

# This column will be utilized to asnwer the qustions 3, 4, 5

# Data analysis

## 1 - How many records are there in the database?

In [67]:
count_of_records = df.count()
count_of_records

40839

## 2 - How many distinct home team are there at base?

In [68]:
distinct_home_team = df.select("home_team").distinct().count()
distinct_home_team

309

# 3 - How many times have the home team been victorious?

In [69]:
home_team_win_count = df.filter("match_resul = 'home_team'").count()
home_team_win_count

19864

# 4 - How many times have the away team been victorious?

In [70]:
away_team_win_count = df.filter("match_resul = 'away_team'").count()
away_team_win_count

11544

# 5 - How many matches resulted in a draw?

In [71]:
draw_count = df.filter("match_resul = 'draw'").count()
draw_count

9431

# 6 - How many matches were played in each country?

In [72]:
matches_by_country = df.groupBy("country").count().sort(desc("count"))
matches_by_country.show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|       United States| 1144|
|              France|  801|
|             England|  687|
|            Malaysia|  644|
|              Sweden|  637|
|             Germany|  581|
|              Brazil|  529|
|               Spain|  517|
|            Thailand|  483|
|               Italy|  480|
|         Switzerland|  477|
|             Austria|  475|
|United Arab Emirates|  472|
|        South Africa|  470|
|               Qatar|  467|
|         South Korea|  453|
|           Argentina|  449|
|             Hungary|  431|
|               Chile|  405|
|             Belgium|  396|
+--------------------+-----+
only showing top 20 rows



In [78]:
# matches_by_country.write.csv("matches_by_country", header=True)

# 7 - Which country had the most matches?

In [74]:
country_with_the_most_matches = matches_by_country.limit(1)
country_with_the_most_matches.show()

+-------------+-----+
|      country|count|
+-------------+-----+
|United States| 1144|
+-------------+-----+



In [75]:
name_of_the_country_with_the_most_matches = country_with_the_most_matches.collect()[0]["country"]
name_of_the_country_with_the_most_matches

'United States'

# 8 - Which match had the most goals?

# 9 - Which match had the biggest goal difference?

# 10 - How many matches have taken place in Brazil?